In [3]:
import re
import os
import keras_nlp
import keras

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import pickle

Using TensorFlow backend


In [6]:
dataset = []
with open('filteredpagedata.txt', encoding='utf=8') as f:
    c=0
    files = 0
    entries = 0
    excerpt = ''
    for line in f:
        c+=1
        if '<title>' in line or re.search("=.*?=", line): # Excerpt has finished. Tokenize and Save
            datum = excerpt
            if len(datum) > 32:
                dataset.append(excerpt)
                entries += 1
                if entries % 100 == 0:
                    if entries <= 1000000:
                        file = 'train_complete'
                    else:
                        file = 'val_complete'
                    with open(f'./dataset/txt/{file}.txt', 'a', encoding='utf-8') as outfile:
                        for entry in dataset:
                            outfile.write(entry)
                            outfile.write('\n<eob>\n')
                        dataset = []
                        
                    #<eob>
            excerpt = ''
        else:
            excerpt += line
        if c%1000000000 == 0:
            break
            print(f'Lines Processsed: {c}, Data Points: {len(dataset)}')

In [1]:
# Data
BATCH_SIZE = 64
MIN_STRING_LEN = 512  # Strings shorter than this will be discarded
SEQ_LEN = 512  # Length of training sequences, in tokens
VOCAB_SIZE = 50000

In [4]:
raw_train_ds = (
    tf_data.TextLineDataset('./dataset/wikitext-103/wiki.train.tokens')
    .filter(lambda x: tf_strings.length(x) > MIN_STRING_LEN)
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

In [5]:
# Takes like 45 minutes. Beware.
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

In [7]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

In [8]:
with open('./models/wikitext103_tokenizer50k.pkl', 'wb') as out:
    pickle.dump(tokenizer, out, protocol=pickle.HIGHEST_PROTOCOL)